### A Note on Saving Data
Web scrapers face a unique set of challenges, the most interesting of which may be the adversarial nature of it all. In general, webmasters do not want their data scraped, and they have a handful of tools available to them to inconvenience the webscraper. The site from which we'll be scraping data, profootballreference, deters webscrapers by temporarily blocking traffic from users that make too many requests in too short of a time period. To counteract this, we'll make use of python's `time.sleep()` function, but, where possible, we'll also be saving webpages locally. This will allow us ad hoc access without the need to wait through numerous calls to `time.sleep()`.

### Data Collection

Naturally, we must begin with collecting the data. This will involve scraping profootballreference.

Let's define our imports and some helper functions to keep our code organized.

In [ ]:
import time
import tqdm
import pandas as pd
import html5lib


def get_passing_url(year):
    return f"https://www.pro-football-reference.com/years/{year}/passing.htm"


def get_rushing_url(year):
    return f"https://www.pro-football-reference.com/years/{year}/rushing.htm"


def get_receiving_url(year):
    return f"https://www.pro-football-reference.com/years/{year}/receiving.htm"


def get_soup(url):
    import requests
    from bs4 import BeautifulSoup
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def get_valid_subdirectories(soup):
    html_subdirectories = soup.find_all("a")
    valid_subdirectories = [link for link in html_subdirectories if '/players/' in str(link)]
    subdirectories = [link.get('href') for link in valid_subdirectories]
    return subdirectories


def make_directory(name):
    import os
    if not os.path.exists(name):
        os.makedirs(name)
        
        
def save_html(soup, name):
    with open(name, "w") as file:
        file.write(str(soup))

Great. Now our first step will involve iterating through each season of interest and getting the webpages for that season's passing, rushing, and receiving stats. We will be saving those pages locally.



#### A Note on Seasons

How many seasons should would be interested in? That's a difficult question to answer. The naive perspective suggest that we want *all* of the available data, and, while there would be some merit to that approach, anyone familiar wih the NFL knows that the game has evolved over time; the way that the game was played in its earliest form has little in common with the game as it exists today, and historical data would feature trends that are not mirrored today. For that reason, we want to focus on "modern" data.

Still, defining 'modern" is not easy. When discussing the NFL, "modern" generally refers to any season after the NFL-AFL merger in 1970. In most contexts, this is a good defintion, but, for the purpose of this project, I don't feel that it is sufficient. The way that the game is played and the ways in which players are utilized continued to evolve after the merger, and the data reflects that. For that reason, I had to settle on a different defintion of "modern."

For the purposes of this project, the modern NFL began in 2012. In 2011, the union representing the players (the NFL Player's Association) negotiated a new collective bargaining agreement (CBA) with the team owners. The CBA had *massive* ramifications, many of which have surely gone unnoticed, but one of the more visible effects has been the reduced interest in signing veteran players to be minor contributors. To oversimplify a nuanced and fascinating topic: the CBA that took prior to the 2012 season limited earnings for rookie players, increasing their value relative to their veteran counterparts. The demand for low-level veterans plummted because rookie contract players offer 90% of the performance for 50% of the price (numbers that I've fabricated to demonstrate the point). This shift meant that many positions (especially runningback) skewed much younger after the CBA took effect. For this project, we want data that reflects current positional values, thus, we are using only data from after the 2011 CBA.

In [ ]:
make_directory("data/season/passing")
make_directory(f"data/season/rushing")
make_directory(f"data/season/receiving")

for year in tqdm.tqdm(range(2012, 2023)):
    # Passing
    passing_url = get_passing_url(year)
    passing_soup = get_soup(passing_url)
    save_html(passing_soup, f"data/season/passing/{year}.html")
    # Rushing
    rushing_url = get_rushing_url(year)
    rushing_soup = get_soup(rushing_url)
    save_html(rushing_soup, f"data/season/rushing/{year}.html")    
    # Receiving
    receiving_url = get_receiving_url(year)
    receiving_soup = get_soup(receiving_url)
    save_html(receiving_soup, f"data/season/receiving/{year}.html")
    time.sleep(30)

With the season stats pages saved, we now need to parse them in order to extract a list of links to the pages for every player that has recorded a pass, rush, or catch since 2011. From those links, we must save the corresponding webpage to file.

In [24]:
import os
from bs4 import BeautifulSoup

data_dir = "data/season"

# Get the list of file paths
file_paths = []
for dir in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, dir)):
        for file in os.listdir(os.path.join(data_dir, dir)):
            if file.endswith(".html"):
                file_paths.append(os.path.join(data_dir, dir, file))
                
# Convert each file to soup
soups = [BeautifulSoup(open(file_path, "r").read(), 'html.parser') for file_path in file_paths]

# Get the list of player subdirectories
player_links = []
for soup in soups:
    # Passing
    passing_table = passing_soup.find(id="passing")
    passing_subdirectories = get_valid_subdirectories(passing_table)
    player_links.extend(passing_subdirectories)
    # Rushing
    rushing_table = rushing_soup.find(id="rushing")
    rushing_subdirectories = get_valid_subdirectories(rushing_table)
    player_links.extend(rushing_subdirectories)
    # Receiving
    receiving_table = receiving_soup.find(id="receiving")
    receiving_subdirectories = get_valid_subdirectories(receiving_table)
    player_links.extend(receiving_subdirectories)
    
# Make unique
player_links = list(set(player_links))

# Get player soups and save to file
make_directory("data/players")
for link in tqdm.tqdm(player_links):
    url = f"https://www.pro-football-reference.com{link}"
    soup = get_soup(url)
    filename = link.split("/")[-1]
    save_html(soup, f"data/players/{filename}")
    time.sleep(5)

100%|██████████| 614/614 [56:59<00:00,  5.57s/it]


Great, we now have a list of subdirectories for every player that has passed, rushed, or caught a ball since 2011. We can append these subdirectories to a base URL to get links to each player's page.

Next we'll have to parse the HTML of each player's page and extract the data. Because doing so would involve several layers of data structure nesting, I opted instead to create a `Player` class to hold each player's data. Because "Tell, Don't Ask" programming is a good practice, and because we are good little object-oriented programmers, we'll instantiate the class with just a single subdirectory and have it handle the HTML parsing. Unfortunately, the format of each player's webpage varies depending on the player's position, meaning we need position-specific parsing functionality, and, thus, we will have `Player` be an abstract class, and we'll create position classes.

In [15]:
from abc import ABC, abstractmethod
from bs4 import BeautifulSoup
import requests


class Player(ABC):
    def __init__(self, link):
        self.url = f"https://www.pro-football-reference.com{link}"
        self.soup = self.get_soup()

    def get_soup(self):
        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup

    @abstractmethod
    def some_player_method(self):
        pass
    
    @classmethod
    def create(cls, link):
        position = cls._get_position_static(link)
        if position == 'QB':
            return Quarterback(link)
        else:
            return SkillPosition(link)

    @staticmethod
    def _get_position_static(link):
        url = f"https://www.pro-football-reference.com{link}"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        position_strong_tag = soup.find('strong', string='Position')
        if position_strong_tag and position_strong_tag.next_sibling:
            position = position_strong_tag.next_sibling.strip(': ').strip()
            return position
        return None


class Quarterback(Player):
    def __init__(self, link):
        super().__init__(link)
        self._set_throws()
        
    def some_player_method(self):
        print('Quarterback method')

    def _set_throws(self):
        self.throws = self.get_throws()

    def get_throws(self):
        throws_strong_tag = self.soup.find('strong', string='Throws:')
        if throws_strong_tag and throws_strong_tag.next_sibling:
            throws = throws_strong_tag.next_sibling.strip(': ').strip()
            return throws
        return None
    
class SkillPosition(Player):
    def some_player_method(self):
        # Implementation specific to SkillPosition
        print('SkillPosition method')

In [17]:
player_link = player_links[0]
player = Player.create(player_link)